# (Optional) Ungraded Lab - Deploying a Deep Learning Model with Docker

In this lab, you will package the code you built earlier via [Docker](https://www.docker.com/). It will **require you to run the code in your local machine as this is not currently possible in the Coursera environment**. If you're already familiar with Docker or have some issues running the code here on your computer, feel free to skip this notebook and select the `cloud-deploy.ipynb` notebook in the file explorer on the left.

### Why Docker?

Docker is an amazing tool that allows you to **ship your software along with all of its dependencies**. This is great because it enables you to run software even without installing the required interpreters or compilers for it to run. 

Let's use an example to explain this better: 

Suppose you trained a model using Python along with some libraries such as Tensorflow or JAX. For this, you created a virtual environment in your local machine. Everything works fine but now you want to share this model with a colleague who does not have Python installed, much less any of the required packages. 

In a pre-Docker world, your colleague would have to install all of this software just to run your model. Instead, by installing Docker, you can share a Docker image that includes all your software and that will be all that is needed.

### Some key concepts

You just read about Docker images and might be wondering what they are. Here are three key concepts to understand how Docker works. These are **Dockerfiles**, **images** and **containers**, and will be explained in this order as each one uses the one before it:

- `Dockerfile`: This is a special file that contains all of the instructions required to build an image. These instructions can be anything from "install Python version 3.7" to "copy my code inside the image".

- `Image`: This refers to the collection of all your software in one single place. Using the previous example, the image will include Python, Tensorflow, JAX and your code. This will be achieved by setting the appropriate instructions within the Dockerfile.

- `Container`: This a running instance of an image. Images by themselves don't do much aside from saving the information of your code and its dependencies. You need to run a container out of them to actually run the code within. Containers are usually meant to perform a single task but they can be used as runtimes to run software that you haven't installed.

Now that you have a high level idea of how Docker works, it is time for you to install it. If you already have it installed you can skip most of the following items.

## Installation

Visit this [link](https://www.docker.com/products/docker-desktop) to install the free version of Docker.

### Note for Windows users

If you are currently on a Windows machine, we recommend you install [WSL2](https://docs.microsoft.com/en-us/windows/wsl/install-win10), in particular a Debian-based distribution such as **Ubuntu**. This will enable a real Linux kernel inside your Windows OS and will get full usage out of Docker and other command line tools you will be using such as [curl](https://curl.se/). **Machine learning deployment is usually done in Linux machines on the cloud so it is good that you get to know this OS for the purpose of deploying your models**.

If you already have WSL2 installed, check out this [link](https://docs.docker.com/docker-for-windows/wsl/) to complete the Docker Desktop installation.

**The commands below should be run using a WSL2 shell.** To open one, use the Windows search bar and type either `wsl` or `bash`. One of these should be available if you installed WSL2 previously.

You may also like to install the [Windows Terminal](https://docs.microsoft.com/en-us/windows/terminal/get-started) since it allows for a better experience to deal with all your shells and command lines.

## Testing the Docker installation

To test your installation go into the command line and type the following command:

```bash
docker run hello-world
```

This command will try to run the `hello-world` image. If you don't have this image yet, Docker will look for it in the [Docker Hub](https://hub.docker.com/) and pull it to your machine to run it. If everything worked well, you should see the following output printed in your command line:

```
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

```

If the command didn't work, check if you encountered errors in your installation, then run the command again.

## Download the Application

Download the zipfile from [this link](https://storage.googleapis.com/mlep-public/course_1/week1/mlepc1w1_cloud.zip). It contains all the files needed to setup your FastAPI application via a Docker container. Unzip the file to your local storage and navigate to it using the command line. Here are the contents:

* (hidden directory) `.cvlib` - This contains a `yolov3-tiny` model. It will be used as the default model of your application.
* `app/main.py` - This is basically the same API endpoints code that you used in the Jupyter notebooks earlier. 
* `app/__init__.py` - This file is just to indicate that the `app` folder is a module
* `Dockerfile` - Sets up the application
* `requirements.txt` - Python packages needed to run the app.

## Build the image

From the directory that contains the unzipped files, type this command:

```
docker build -t w1_lab .
```

This will build an image called `w1_lab` based on the Dockerfile provided. You can see what each command does [here](https://docs.docker.com/reference/dockerfile/). In short, it will contain all the packages, code, and model required to run your object detection app. If you want to know more about setting up a FastAPI app via Docker, you can look [here](https://fastapi.tiangolo.com/deployment/docker/). It will take a several minutes to finish building the image.

## Run a container

Next, you will run a container called `w1_lab` based off the image you built (which is also named `w1_lab`). It will run in the background as specified by the `-d` flag and you can access it on port 80 ([see here](https://docs.docker.com/get-started/docker_cheatsheet.pdf) for other frequently used docker commands):

```
docker run -d --name w1_lab -p 80:80 w1_lab # use some other port if 80 is already in use
```

It should output a long hash string indicating that it succeeded. Type this command to see running containers:

```
docker ps
```

You should see something like this:

```
CONTAINER ID   IMAGE     COMMAND                  CREATED          STATUS          PORTS                NAMES
2dad91e993d6   w1_lab    "uvicorn app.main:ap…"   16 seconds ago   Up 15 seconds   0.0.0.0:80->80/tcp   w1_lab
```

This should let you know that the app is now up and running.

## Test the App

Open a web browser and go to `http://localhost:80`. You should see the same familiar message:

<img src='assets/docker/localhost.png' width='500'>

<br>
<br>


Appending `/docs` will also land you in the FastAPI UI:

<img src='assets/docker/localhostdocs.png' width='500'>

<br>
<br>

Feel free to upload some images to see that the object detection is working.

## Managing the container

After testing, you can stop the service with:

```
docker stop w1_lab
```

You should not see it in the output when you run `docker ps` again. You can restart the container with:

```
docker start w1_lab
```

If for some reason you want to remove it completely, you can run this while the container is stopped:

```
docker rmi w1_lab #kill or remove container first
```

## Wrap Up

In this exercise, you saw how you can package your FastAPI object detection model application via Docker. You're able to do so with a container that has all the necessary packages and code to serve your deep learning model. This will come in handy when you want to deploy your app on different devices or networks. You can see one such example in the next optional lab that runs this same Docker application in the Cloud. If you're up for it, double click on the `cloud-deploy.ipynb` notebook in the file explorer on the left.